# Collect the Observed Snow Data using CUAHSI Data Client

This noteook retrieves Snow Water Equivalent (SWE) and accumulated precipitation (P) data from SNOTEL sites through CUAHSI data client service.  The notebook has five sections as following:

1. Import Libraries
2. Visualize Ecoregions
3. Define Parameters
4. Define Functions 
5. Data Acquisition

## 1. Import Libraries
In this section, required python libraries are installed and imported. 

In [8]:
import os
import ulmo  
import pytz
import datetime
from timezonefinder import TimezoneFinder
import matplotlib
import numpy as np
import pandas as pd
from pylab import *
import glob
from matplotlib import colors
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from datetime import datetime
from suds.client import Client
import matplotlib.pyplot as plt
from pandas import Grouper  
from datetime import timedelta
import matplotlib.dates as mdates

%matplotlib inline

## 2.   Read SNOTEL Information
Information includes but not limited to station code, station latitude and longitude, and associated ecoregions provided by the Commission for Environmental Corporation (CEC).

In [9]:
snotel_network = '../input/NRCS_SNOTEL_Joint_w_CEC.csv'  
output_dir = '../output'
snotel_info = pd.read_csv(os.path.join(os.getcwd(), snotel_network))
snotel_info = snotel_info[snotel_info['Join_Count'] == 1]                 # To make sure only those associated with CEC are used

## 3.   Define Parameters

Following parameters identify the name of the network from which we want to retrieve data. It also creates the CUAHSI link to the defined network. 

In [10]:
# Start preparing input arguments that are required when connecting to CUAHSI data client service
network = "SNOTEL"     
wsdl = f'http://hydroportal.cuahsi.org/{network.lower()}/cuahsi_1_1.asmx?WSDL'
client = Client(wsdl)

## 4.   Define Functions
Two functions are defined in this Jupyter Notebook. *DataFrame* and *Local2UTC*.  The first function is used to retrieve SNOTEL data.  The second function is used when we want to transform local time to UTC time.  

- **DataFrame**: This function creates a dataframe for retrieved data including retrieved information. 


- **Local2UTC**: There are two different time zones within these ecoregions, Mountain Time and Pacific Time zones.  'America/Phoenix', is in the Mountain Time zone and does not observe daylight saving time.  It always has 7 hours offset from UTC.  'America/Boise' and 'America/Denver' are also in the Mountain Time zone with 6 hours offset from UTC during daylight saving time and 7 hours offset in standard time.  'America/Los_Angeles' is in the Pacific Time with 7 hours offset from UTC during daylight saving time and 8 hours ofset in standard time.  These are important when comparing daily average values of SNOTEL with hourly datasets such as NWM where outputs are in UTC time or SNODAS snapshots that are reported at 6:00 UTC.  The Local2UTC function takes a dataframe that has a column called 'date' (showing the local time for each record (row)) and a list of integer indices of the dataframe as inputs.  The function runs over each row of the dataframe and reads corresponding latitude and longitude values to get the name of the time zone ('America/Phoenix', America/Boise', 'America/Denver', or 'America/Los_Angeles').  Then, it uses pytz library and and transform the local date/time to a UTC date/time.  The next part checks for the offset value and updates the time for the local time that by default is 00:00:00.  This update helps to choose a time for SNOTEL local time that is equivalent to 6:00 UTC (as used in SNODAS outputs).


- **Data_Retrieval**: This function uses snotel information (csv file), DataFrame, and Local2UTC functions to retrieve SNOTEL data from CUAHSI data client service. 

In [11]:
# Define a function to make a dataframe from retrieved data.  
# This function can be used for both SWE and Precipitation retrievals.

def DataFrame(self, st, et):
    
    # series info
    qo = self.queryInfo
    self.site_code = qo.criteria.locationParam
    self.variable_code = qo.criteria.variableParam
    self.start = st 
    self.end = et 

    # source info
    si = self.timeSeries[0].sourceInfo
    self.site_name = si.siteName
    self.latitude = si.geoLocation.geogLocation.latitude
    self.longitude = si.geoLocation.geogLocation.longitude
    
    # variable
    v = self.timeSeries[0].variable
    self.variable_name = v.variableName
    self.variable_datatype = v.dataType
    self.units_abbv = v.unit.unitAbbreviation
    self.nodata = v.noDataValue

    # values
    self.data = []
    for val in self.timeSeries[0].values[0].value:
        value_dt  = val._dateTime
        value_in = float(val.value)
        if value_in != self.nodata:
            value_mm = value_in * 25.4
        else:
            value_in = np.NaN
            value_mm = np.NaN
        self.data.append(dict(date=value_dt,
                              value_inches=value_in,
                              value_mm=value_mm))
     
    atts = {k:v for k,v in self.__dict__.items()}

    dat = []
    for val in self.data:
        content = {k:v for k,v in atts.items()}
        for k, v in val.items():
            content[k] = v
        dat.append(content)  

    df = pd.DataFrame(dat)
    df = df.set_index(df.date)
    
    return df


In [12]:
# Define a function that transform local time to UTC.

def Local2UTC(data, indices):
    
    tf = TimezoneFinder()

    LOCAL = []
    UTC = []
    
    # Loop over each row of the dataframe    
    for row in indices:
                
        timezone_str = tf.timezone_at(lng=data.longitude[row], lat=data.latitude[row])
        local_time = pytz.timezone(timezone_str)
        # The next line uses the date and by default gives the time 00:00:00 
        # (i.e., 12:00 am) to the datetime object
        naive_datetime = datetime.strptime(data['date'][row].strftime('%Y-%m-%d %H:%M:%S'), "%Y-%m-%d %H:%M:%S") 
        local_datetime = local_time.localize(naive_datetime, is_dst=None)
        utc_datetime = local_datetime.astimezone(pytz.utc)
        
        # Update naive_time based on timezone. This update helps to choose a 
        # time for SNOTEL daily values that are equivalent to 6:00 am UTC
        
        # Condition 1: if (local_time == "America/Phoenix") or 
        # (local_time == "America/Denver" and utc_datetime.hour == 7, Standard) or
        # (local_time == "America/Boise" and utc_datetime.hour == 7, Standard) or
        # (local_time == "America/Los_Angeles" and utc_datetime.hour == 7, Daylight)
        if utc_datetime.hour == 7:
            naive_datetime = naive_datetime + timedelta(hours=23)
            local_datetime = local_time.localize(naive_datetime, is_dst=None)
            utc_datetime = local_datetime.astimezone(pytz.utc)
            utc_datetime_str = utc_datetime.strftime ("%Y-%m-%d %H:%M:%S")
            
        # Condition 2: if (local_time == "America/Denver" and utc_datetime.hour == 6, Daylight) or
        # (local_time == "America/Boise" and utc_datetime.hour == 6, Daylight)
        elif utc_datetime.hour == 6: 
            naive_datetime = naive_datetime
            local_datetime = local_time.localize(naive_datetime, is_dst=None)
            utc_datetime = local_datetime.astimezone(pytz.utc)
            utc_datetime_str = utc_datetime.strftime ("%Y-%m-%d %H:%M:%S")
            
        # Condition 3: if (local_time == "America/Los_Angeles" and utc_datetime.hour == 8, Standard)
        elif utc_datetime.hour == 8: 
            naive_datetime = naive_datetime + timedelta(hours=22)
            local_datetime = local_time.localize(naive_datetime, is_dst=None)
            utc_datetime = local_datetime.astimezone(pytz.utc)
            utc_datetime_str = utc_datetime.strftime ("%Y-%m-%d %H:%M:%S")
            
        
        LOCAL.append(local_datetime)
        UTC.append(utc_datetime_str)
        
        
    # Add columns to the dataframe
    data['datetime_LOCAL'] = LOCAL
    data['datetime_UTC'] = UTC
        
    # Add UTC times to index
    data.index = data['datetime_UTC']

    return data


In [13]:
# Define a function that retrieves data from CUAHSI data services.

def Data_Retrieval(info_dataframe, variable, abbr, output_dir, output_name):
    
    site_code = info_dataframe['Station_ID']
    variable_code_daily = f'{network}:{variable}'
    var_Daily = pd.DataFrame([])
    
    
    for i in range(0, len(site_code)):
        
        code = f'{site_code[0]}_{info_dataframe["State"][0]}_SNTL'
        sitecode = f'{network}:{code}'
        site = ulmo.cuahsi.wof.get_site_info(wsdl, sitecode, suds_cache=('default', ))    
        sc = f'{network}:{site["code"]}'   
        st = datetime.strptime(site['site_property']['site_comments'].split('|')[0].split('=')[1].split(' ')[0], '%m/%d/%Y')  
        et = datetime.strptime(site['site_property']['site_comments'].split('|')[1].split('=')[1].split(' ')[0], '%m/%d/%Y')
    
    
        try:
            temp = client.service.GetValuesObject(sc, variable_code_daily, st, et, '')
            # Use DataFrame function defined above
            temp_df = DataFrame(temp, st, et)
            var_Daily = var_Daily.append(temp_df, ignore_index=True)
            print(i, sc, st, et)  
        except Exception as error:
            print("==========================================================")
            print(f'{variable} at {sc} with index {i} in site_code is not available.')
            print("==========================================================")
            pass
        
        
    Daily_small_df = pd.DataFrame({'col1':var_Daily['site_code'], 
                                   'col2':var_Daily['site_name'], 
                                   'col3':var_Daily['date'], 
                                   'col4':var_Daily['value_inches'],
                                   'col5':var_Daily['value_mm'],
                                   'col6':var_Daily['latitude'],
                                   'col7':var_Daily['longitude']}) 
              
    
    Daily_small_df.columns = ['site_code', 'site_name', 'date', f'{abbr}_inches', 
                               f'{abbr}_mm', 'latitude', 'longitude']


    info_dataframe = info_dataframe.reset_index()
    for c in range (0, len(site_code)):
        Daily_small_df.loc[Daily_small_df['site_name'] == info_dataframe['Station_Na'][c], 'name'] = info_dataframe['NAME'][c]
        
        
    # Run Local2UTC function defined above
    Daily_small_df_utc = Local2UTC(Daily_small_df, Daily_small_df.index)
        
        
    # Save results as a CSV file
    Daily_small_df_utc.to_csv(f'{output_dir}/{output_name}', index=False)

     

## 5.   Data Acquisition

Following retrieves all available snow water equivalent (SWE) and precipitation data for all gages.

In [ ]:
# Data_Retrieval(snotel_info, 'WTEQ_D', 'swe', output_dir, 'SNOTEL_SWE.csv')
Data_Retrieval(snotel_info, 'PREC_D', 'precip', output_dir, 'SNOTEL_P.csv')

0 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
1 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
2 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
3 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
4 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
5 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
6 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
7 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
8 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
9 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
10 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
11 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
12 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
13 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
14 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
15 SNOTEL:301_CA_SNTL 1983-10-01 00:00:00 2100-01-01 00:00:00
16 SNOTEL:301_CA_S